# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Alexandre Vielfaure
    - Hatim Belgharbi
    - Johan Ngounou




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [90]:
import numpy as np
import math
import copy
import pandas as pd
import random
from collections import deque

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        self.rock = ()

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.nb_moves = self.nb_moves + 1
        s.rock = self.rock
        # TODO
        return s

    def put_rock(self, rock_pos):
        

        temp_pos = self.pos.copy()
        new_state = State(temp_pos)
        new_state.prev = self
        new_state.c = self.c
        new_state.d = self.d
        new_state.nb_moves = self.nb_moves + 1
        new_state.rock = rock_pos
        
        return new_state
            
    
    def which_car(self,rh,target_line,target_column):
        for car in range(rh.nbcars):
            if (not rh.horiz[car]): # vertical cars
                for line in range(rh.length[car]):
                    if((self.pos[car]+line == target_line) and (rh.move_on[car] == target_column)):
                        return car
                        break
            else:   # horizontal cars
                for column in range(rh.length[car]):
                    if((self.pos[car]+column == target_column) and (rh.move_on[car] == target_line)):
                        return car
                        break
        return -1
    
    def is_block(self,c,posb,rh):
    
        c=c
        posb=posb
        
        n = rh.length[c]
        rg = range(6)
        poss_rang = list(zip(*(rg[i:] for i in range(n))))
        ar = []
        for i in poss_rang:
            ar.append(abs(i[0] - self.pos[c]))
        sort_rang = [x for _,x in sorted(zip(ar,poss_rang))]
        
        test = ()
        for i in sort_rang:
            if not (posb in i):
                test = i
                break
        #print(test)
        dis = abs(test[0]-self.pos[c])
        
        new_block = []
        for i in test:
            if not rh.horiz[c]:
                blocking_car = self.which_car(rh,i,rh.move_on[c])
            else: 
                blocking_car = self.which_car(rh,rh.move_on[c],i)
                
            if (blocking_car != c) and (blocking_car != -1):
                if rh.horiz[blocking_car] == rh.horiz[c]:
                    ff = set(test).intersection(set(range(self.pos[blocking_car],self.pos[blocking_car]+rh.length[blocking_car]))).pop()
                    new_block.append((blocking_car,ff))
                else:
                    new_block.append((blocking_car,rh.move_on[c]))
               
        return [new_block,dis,test[0]]
    
    def score_state(self,rh):
        
        cost = (4 - self.pos[0] + self.nb_moves)
        rh.init_positions(self)
        
        for i in range(rh.nbcars):
            if (not rh.horiz[i]) and (rh.move_on[i] > self.pos[0]+1):
                line_occupied = range(self.pos[i],self.pos[i]+rh.length[i])
                if 2 in line_occupied:
                    new_c = [(i,2)]
                    compt = 0
                    visited = []
                    while new_c:
                        #print(new_c)
                        val = new_c.pop()
                        c = val[0]
                        if not c in visited:
                            visited.append(c)
                            
                            posb= val[1]

                            block_val = self.is_block(c,posb,rh)
                            new_c += block_val[0]
                            
                            #If a 3 length blocking car in the loop, add weight
                            if new_c and not rh.horiz[c] and rh.length[c] == 3 and rh.move_on[c] > self.pos[0]+1:
                                compt +=2
                                
                            dis = block_val[1]
                            
                            compt = compt+dis+3
                            
                            if self.pos[c] - self.prev.pos[c] == 0 : #If you don't move a blocking car
                                compt += 1
                            
                            #If you move a blocking car in a position that cannot move
                            pos_car = 0
                            for j in rh.possible_moves(self) :
                                if self.pos[c] - j.pos[c] != 0:
                                    pos_car += 1
                            if pos_car == 0 :
                                pos_car = -20
                                
                            compt += (2-pos_car)
                    
                    cost += compt
                    
                    if rh.length[i] == 3: #If a blocking car of red of length 3 is not in good position 
                        if (3-self.pos[i]) > 0:
                            cost += 1      
        #Prevent cycles
        if self.prev.c:
            if (self.prev.c - self.c) == 0:
                if (self.prev.d + self.d) == 0 :
                    cost +=1000
                    
        self.score = cost
        

    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [91]:
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
        
        if state.rock:
            self.free_pos[state.rock] = False

    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1))
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
                
        for i in range(len(self.free_pos)):
            for j in range(len(self.free_pos.T)):
                
                if (not state.rock):
                    if (self.free_pos[i,j]) == True  and (i!=2):
                        new_states.append(state.put_rock((i,j)))                     
                else :
                    if (self.free_pos[i,j]) == True and (i!=state.rock[0]) and ((j!=state.rock[1])) and (i!=2):
                        new_states.append(state.put_rock((i,j)))
          
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock:
            grid[state.rock] = 'x'
        print(grid)
        

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [92]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [93]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    rh.print_pretty_grid(s)
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [94]:
class MiniMaxSearch:
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self.search_car = search_depth
        self.search_rock = 2

    def minimax_1(self, current_depth, current_state): 
        
        if current_depth == 0  or current_state.success():
            current_state.score_state(self.rushhour)
            return current_state
        
        v = State([0])
        v.score = np.inf
        
        for successor in self.rushhour.possible_moves(current_state):
            if np.random.randint(2):
                v=min(v,self.minimax_1(current_depth - 1,successor))
            else:
                v= min(self.minimax_1(current_depth - 1,successor),v)
            
        best_move = v
        
        if (self.search_depth - current_depth) >= 1:
            new_best_move = best_move.prev
            new_best_move.score = best_move.score  
        else:
            new_best_move = best_move

        return new_best_move
    
    def minimax_2(self, current_depth, current_state, is_max): 
        global nb_state
        if current_depth == 0  or current_state.success():

            #print(current_state.rock)
            current_state.score_state(self.rushhour)
            nb_state = nb_state + 1
            return current_state
        
        v = State([0])
        
        if not is_max:
            v.score = np.inf
            for successor in self.rushhour.possible_moves(current_state):
                
                if np.random.randint(2):
                    v=min(v,self.minimax_2(current_depth - 1,successor,not is_max))
                else:
                    v=min(self.minimax_2(current_depth - 1,successor,not is_max),v)
        else:
            v.score = -1*np.inf
            #self.rushhour.print_pretty_grid(current_state)
            for successor in self.rushhour.possible_rock_moves(current_state):
                #self.rushhour.print_pretty_grid(successor)
                if np.random.randint(2):
                    v=max(v,self.minimax_2(current_depth - 1,successor,not is_max))
                else:
                    v= max(self.minimax_2(current_depth - 1,successor,not is_max),v)
         
        best_move = v
        
   
        if (self.search_depth - current_depth) >= 1:
            new_best_move = best_move.prev
            new_best_move.score = best_move.score  
        else:
            new_best_move = best_move

        return new_best_move
        
    
    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        global nb_state
        if current_depth == 0  or current_state.success():

            current_state.score_state(self.rushhour)
            nb_state = nb_state + 1
            return current_state
        
        v = State([0])
        
        if not is_max:
            v.score = np.inf
            for successor in self.rushhour.possible_moves(current_state):
                if np.random.randint(2):
                    v=min(v,self.minimax_pruning(current_depth - 1,successor,not is_max, alpha, beta))
                else:
                    v=min(self.minimax_pruning(current_depth - 1,successor,not is_max, alpha, beta),v)
                
                if v.score <= alpha: # Pruning
                    break
                beta = min(beta,v.score)
        else:
            v.score = -1*np.inf
            for successor in self.rushhour.possible_rock_moves(current_state):
                if np.random.randint(2):
                    v=max(v,self.minimax_pruning(current_depth - 1,successor,not is_max, alpha, beta))
                else:
                    v= max(self.minimax_pruning(current_depth - 1,successor,not is_max, alpha, beta),v)
                if v.score >= beta: # Pruning
                    break
                alpha = min(alpha,v.score)
         
        best_move = v
     
        if (self.search_depth - current_depth) >= 1:
            new_best_move = best_move.prev
            new_best_move.score = best_move.score  
        else:
            new_best_move = best_move

        return new_best_move
        


    def expectimax(self, current_depth, current_state, is_max):
        global nb_state
        global option
        if current_depth == 0  or current_state.success():

            current_state.score_state(self.rushhour)
            nb_state = nb_state + 1
            return current_state
        
        v = State([0])

        if not is_max:
            v.score = np.inf
            for successor in self.rushhour.possible_moves(current_state):
                if np.random.randint(2):
                    v=min(v,self.expectimax(current_depth - 1,successor,not is_max))
                else:
                    v=min(self.expectimax(current_depth - 1,successor,not is_max),v)
        else: # expectimax
            if option == 0:
            # alea
                v = self.rushhour.possible_rock_moves(current_state)[0]
                v.score = 0
                for successor in self.rushhour.possible_rock_moves(current_state):
                    p = 1/len(self.rushhour.possible_rock_moves(current_state))
                    v.score = v.score + p*(self.expectimax(current_depth - 1,successor,not is_max).score)             
            elif option == 1:
            # pessimiste
                v = self.rushhour.possible_rock_moves(current_state)[0]
                v.score = 0
                maxim = -np.inf
                for successor in self.rushhour.possible_rock_moves(current_state):
                    p = 1/len(self.rushhour.possible_rock_moves(current_state))
                    maxim = max(maxim,self.expectimax(current_depth - 1,successor,not is_max).score)
                for successor in self.rushhour.possible_rock_moves(current_state):
                    p = 1/len(self.rushhour.possible_rock_moves(current_state))
                    # On favorise les grandes valeurs
                    k = self.expectimax(current_depth - 1,successor,not is_max).score / maxim # paramètre de modification de la probabilité
                    v.score = v.score + p*k*(self.expectimax(current_depth - 1,successor,not is_max).score)
            # optimiste
            else:
                v = self.rushhour.possible_rock_moves(current_state)[0]
                v.score = 0
                maxim = -np.inf
                for successor in self.rushhour.possible_rock_moves(current_state):
                    p = 1/len(self.rushhour.possible_rock_moves(current_state))
                    maxim = max(maxim,self.expectimax(current_depth - 1,successor,not is_max).score)
                for successor in self.rushhour.possible_rock_moves(current_state):
                    p = 1/len(self.rushhour.possible_rock_moves(current_state))
                    # On favorise les petites valeurs
                    k = self.expectimax(current_depth - 1,successor,not is_max).score / maxim
                    v.score = v.score + p/k*(self.expectimax(current_depth - 1,successor,not is_max).score)
            
        best_move = v
     
        if (self.search_depth - current_depth) >= 1:
            new_best_move = best_move.prev
            new_best_move.score = best_move.score  
        else:
            new_best_move = best_move

        return new_best_move
        
    

    def decide_best_move_1(self):
        
        self.state = self.minimax_1(self.search_depth,self.state)
                
        return 

    def decide_best_move_2(self, is_max):

        if not is_max:
            self.search_depth = self.search_car
            self.state = self.minimax_2(self.search_depth,self.state,is_max)
        else:
            self.search_depth = self.search_rock
            self.state = self.minimax_2(self.search_depth,self.state,is_max)
        
        return
        

    def decide_best_move_pruning(self, is_max):
        
        if not is_max:
            self.search_depth = self.search_car
            self.state = self.minimax_pruning(self.search_depth,self.state,is_max, -np.inf, np.inf)
        else:
            self.search_depth = self.search_rock
            self.state = self.minimax_pruning(self.search_depth,self.state,is_max, -np.inf, np.inf)
        
        return

    def decide_best_move_expectimax(self, is_max):
        
        if not is_max:
            self.search_depth = self.search_car
            self.state = self.expectimax(self.search_depth,self.state,is_max)
        else:
            self.search_depth = self.search_rock
            all_moves = self.rushhour.possible_rock_moves(self.state)
            self.state = random.choice(all_moves) # random move
                
        return

    def solve(self, state, is_singleplayer, pruning=False, expectimax=False): # default arguments
        
        if is_singleplayer:
            comp = 0
            is_max = False
            while not self.state.success() and comp < 40:
                if not pruning:
                    self.decide_best_move_1()
                else:
                    self.decide_best_move_pruning(is_max)
                
                #print(self.state.pos)
                self.print_move(False, self.state)
                
                comp = comp + 1
                
        else :
            comp = 0
            is_max = False
            while not self.state.success() and comp < 200:
                if not pruning:
                    self.decide_best_move_2(is_max)
                else:
                    if not expectimax:
                        self.decide_best_move_pruning(is_max)
                    else:
                        self.decide_best_move_expectimax(is_max)

                self.print_move(is_max, self.state)
                
                is_max=not is_max
                
                comp = comp + 1

        return

    def print_move(self, is_max, state):
        global cpt
        new_state = state
        
        if not is_max:

            car = self.rushhour.color[new_state.c]
                        
            if self.rushhour.horiz[new_state.c]:
                cpt = cpt + 1
                if new_state.d > 0:
                    move = 'la droite'
                else:
                    move = 'la gauche'
            else:
                cpt = cpt + 1
                if new_state.d < 0:
                    move = 'le haut'
                else:
                    move = 'le bas'
            print('La voiture '+car+' vers ' + move)
            
        else :
             #Compléter pour roche
            print('Roche : ' + str(new_state.rock))
          


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [95]:
cpt = 0
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1) # Voiture rouge vers la droite

    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Roche : (3, 1)
La voiture rouge vers la droite


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [96]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)


[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
La voiture rouge vers la droite
La voiture orange vers le haut
La voiture jaune vers la gauche
La voiture bleu vers le haut
La voiture vert vers le bas
La voiture vert vers le bas
La voiture vert vers le bas
La voiture rouge vers la droite
La voiture rouge vers la droite


In [97]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 1) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
La voiture vert vers la droite
La voiture mauve vers le haut
La voiture orange vers le haut
La voiture lime vers la gauche
La voiture jaune vers le bas
La voiture emeraude vers la gauche
La voiture emeraude vers la gauche
La voiture lime vers la gauche
La voiture lime vers la gauche
La voiture bleu vers le bas
La voiture bleu vers le bas
La voiture jaune vers le bas
La voiture jaune vers le bas
La voiture rouge vers la droite
La voiture rouge vers la droite
La voiture rouge vers la droite


In [98]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,1)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
La voiture 4 vers le haut
La voiture 3 vers le bas
La voiture 2 vers la gauche
La voiture 2 vers la gauche
La voiture 4 vers le haut
La voiture 10 vers la gauche
La voiture rouge vers la droite
La voiture 1 vers le bas
La voiture 3 vers le haut
La voiture 2 vers la gauche
La voiture 3 vers le haut
La voiture 5 vers le bas
La voiture 5 vers le bas
La voiture rouge vers la droite
La voiture rouge vers la droite
La voiture rouge vers la droite


### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [99]:
#Visited state 
tableau = np.zeros((5,3))

# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
nb_state = 0
algo.solve(s, False)
tableau[0,0] = nb_state

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
La voiture rouge vers la droite
Roche : (3, 4)
La voiture bleu vers le haut
Roche : (5, 5)
La voiture vert vers le bas
Roche : (0, 4)
La voiture orange vers le haut
Roche : (5, 1)
La voiture vert vers le bas
Roche : (1, 4)
La voiture jaune vers la gauche
Roche : (5, 0)
La voiture vert vers le bas
Roche : (4, 5)
La voiture rouge vers la droite
Roche : (3, 3)
La voiture rouge vers la droite


In [100]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
nb_state = 0
algo.solve(s, False)
tableau[0,1] = nb_state

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
La voiture vert vers la droite
Roche : (0, 0)
La voiture lime vers la gauche
Roche : (3, 5)
La voiture mauve vers le haut
Roche : (0, 3)
La voiture orange vers le haut
Roche : (3, 5)
La voiture lime vers la gauche
Roche : (4, 1)
La voiture jaune vers le bas
Roche : (5, 5)
La voiture lime vers la gauche
Roche : (4, 3)
La voiture emeraude vers la gauche
Roche : (5, 0)
La voiture bleu vers le bas
Roche : (1, 3)
La voiture emeraude vers la gauche
Roche : (4, 5)
La voiture bleu vers le bas
Roche : (3, 2)
La voiture jaune vers le bas
Roche : (5, 5)
La voiture rouge vers la droite
Roche : (3, 2)
La voiture jaune vers le bas
Roche : (5, 4)
La voiture rouge vers la droite
Roche : (3, 1)
La voiture rouge vers la droite


In [101]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
nb_state = 0
algo.solve(s, False)
tableau[0,2] = nb_state

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
La voiture 4 vers le haut
Roche : (3, 3)
La voiture 2 vers la gauche
Roche : (0, 5)
La voiture 3 vers le bas
Roche : (4, 3)
La voiture 2 vers la gauche
Roche : (0, 4)
La voiture 10 vers la gauche
Roche : (4, 5)
La voiture 4 vers le haut
Roche : (1, 3)
La voiture rouge vers la droite
Roche : (4, 5)
La voiture 1 vers le bas
Roche : (1, 3)
La voiture 2 vers la gauche
Roche : (4, 5)
La voiture 3 vers le haut
Roche : (0, 3)
La voiture 5 vers le bas
Roche : (5, 5)
La voiture 3 vers le haut
Roche : (4, 0)
La voiture 5 vers le bas
Roche : (3, 3)
La voiture rouge vers la droite
Roche : (4, 1)
La voiture rouge vers la droite
Roche : (3, 3)
La voiture rouge vers la droite


## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [102]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s, 3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
#algo.solve(s, False)
nb_state = 0
algo.solve(s, False,True) # testing pruning
tableau[1,0] = nb_state

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
La voiture rouge vers la droite
Roche : (3, 4)
La voiture orange vers le haut
Roche : (5, 1)
La voiture vert vers le bas
Roche : (4, 4)
La voiture jaune vers la gauche
Roche : (0, 5)
La voiture bleu vers le bas
Roche : (4, 4)
La voiture bleu vers le bas
Roche : (5, 5)
La voiture vert vers le bas
Roche : (4, 3)
La voiture vert vers le bas
Roche : (5, 5)
La voiture rouge vers la droite
Roche : (4, 2)
La voiture rouge vers la droite


In [103]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
#algo.solve(s, False) 
nb_state = 0
algo.solve(s, False,True) # testing pruning
tableau[1,1] = nb_state

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
La voiture vert vers la droite
Roche : (0, 0)
La voiture lime vers la gauche
Roche : (3, 5)
La voiture mauve vers le haut
Roche : (0, 3)
La voiture orange vers le haut
Roche : (3, 5)
La voiture lime vers la gauche
Roche : (4, 1)
La voiture jaune vers le bas
Roche : (0, 3)
La voiture lime vers la gauche
Roche : (4, 5)
La voiture emeraude vers la gauche
Roche : (5, 0)
La voiture bleu vers le bas
Roche : (0, 5)
La voiture jaune vers le bas
Roche : (5, 0)
La voiture jaune vers le bas
Roche : (1, 3)
La voiture emeraude vers la gauche
Roche : (5, 4)
La voiture bleu vers le bas
Roche : (3, 2)
La voiture rouge vers la droite
Roche : (1, 5)
La voiture rouge vers la droite
Roche : (0, 3)
La voiture rouge vers la droite


In [104]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
#algo.solve(s, False)
nb_state = 0
algo.solve(s, False,True) # testing pruning
tableau[1,2] = nb_state

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
La voiture 4 vers le haut
Roche : (3, 3)
La voiture 10 vers la gauche
Roche : (4, 5)
La voiture 3 vers le bas
Roche : (0, 2)
La voiture rouge vers la droite
Roche : (4, 5)
La voiture 2 vers la gauche
Roche : (1, 3)
La voiture 2 vers la gauche
Roche : (4, 5)
La voiture 4 vers le haut
Roche : (1, 3)
La voiture 1 vers le bas
Roche : (0, 0)
La voiture 3 vers le haut
Roche : (3, 3)
La voiture 2 vers la gauche
Roche : (4, 5)
La voiture 3 vers le haut
Roche : (3, 4)
La voiture 5 vers le bas
Roche : (5, 5)
La voiture rouge vers la droite
Roche : (4, 1)
La voiture 5 vers le bas
Roche : (3, 4)
La voiture rouge vers la droite
Roche : (4, 1)
La voiture rouge vers la droite


In [109]:
from IPython.display import display, HTML

tableau_frame = pd.DataFrame(tableau,columns = ["9 moves","16 moves","14 moves"],
                               index = ["No Pruning","Pruning","Expectimax Aléatoire"
                                        ,"Expectimax Pessimiste","Expectimax Optimiste"])

print("Visited states :")
display(HTML(tableau_frame.iloc[0:2,:].to_html()))


Visited states :


,9 moves,16 moves,14 moves
No Pruning,4669.0,8162.0,5169.0
Pruning,3301.0,4468.0,2855.0


## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [106]:
#%% Expectimax
tab = np.zeros((3,3))
for option in range(3): # L'option est aléatoire = 0, pessimiste = 1, optimiste = 2
    # Solution optimale: 9 moves
    rh = Rushhour([True, False, False, False, True],
                     [2, 3, 2, 3, 3],
                     [2, 4, 5, 1, 5],
                     ["rouge", "vert", "bleu", "orange", "jaune"])
    s = State([1, 0, 1, 3, 2])
    algo = MiniMaxSearch(rh, s, 3)
    algo.rushhour.init_positions(s)
    print(algo.rushhour.free_pos)
    #algo.solve(s, False)
    nb_state = 0
    cpt = 0
    algo.solve(s, False,True,True) # testing expectimax
    tableau[2+option,0] = nb_state
    tab[option,0] = cpt
    
    
    # solution optimale: 16 moves
    rh = Rushhour([True, True, False, False, True, True, False, False],
                     [2, 2, 3, 2, 3, 2, 3, 3],
                     [2, 0, 0, 0, 5, 4, 5, 3],
                     ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
    s = State([1, 0, 1, 4, 2, 4, 0, 1])
    algo = MiniMaxSearch(rh, s, 3) 
    algo.rushhour.init_positions(s)
    print(algo.rushhour.free_pos)
    #algo.solve(s, False) 
    nb_state = 0
    cpt = 0
    algo.solve(s, False,True,True) # testing expectimax
    tableau[2+option,1] = nb_state
    tab[option,1] = cpt
    # solution optimale: 14 moves

    rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                     [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                     [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                     ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
    s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
    algo = MiniMaxSearch(rh, s,3)
    algo.rushhour.init_positions(s)
    print(algo.rushhour.free_pos)
    #algo.solve(s, False)
    nb_state = 0
    cpt = 0
    algo.solve(s, False,True,True) # testing expectimax
    tableau[2+option,2] = nb_state
    tab[option,2] = cpt

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
La voiture rouge vers la droite
Roche : (3, 4)
La voiture orange vers le haut
Roche : (0, 2)
La voiture jaune vers la gauche
Roche : (4, 5)
La voiture vert vers le bas
Roche : (1, 1)
La voiture vert vers le bas
Roche : (0, 5)
La voiture vert vers le bas
Roche : (1, 4)
La voiture bleu vers le haut
Roche : (4, 5)
La voiture rouge vers la droite
Roche : (3, 0)
La voiture rouge vers la droite
[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
La voiture vert vers la droite
Roche : (1, 1)
La voiture mauve vers le haut
Roche : (4, 3)
La voiture orange vers le haut
Roche : (5, 1)
L

La voiture 4 vers le haut
Roche : (3, 4)
La voiture 3 vers le bas
Roche : (1, 2)
La voiture 2 vers la gauche
Roche : (0, 5)
La voiture 2 vers la gauche
Roche : (1, 3)
La voiture 4 vers le haut
Roche : (5, 5)
La voiture 10 vers la gauche
Roche : (4, 1)
La voiture rouge vers la droite
Roche : (1, 2)
La voiture 1 vers le bas
Roche : (4, 0)
La voiture 3 vers le haut
Roche : (1, 1)
La voiture 2 vers la gauche
Roche : (3, 3)
La voiture 3 vers le haut
Roche : (1, 1)
La voiture rouge vers la droite
Roche : (0, 5)
La voiture 5 vers le bas
Roche : (1, 1)
La voiture 5 vers le bas
Roche : (0, 5)
La voiture rouge vers la droite
Roche : (4, 0)
La voiture rouge vers la droite


In [110]:
print("Visited states :")
display(HTML(tableau_frame.iloc[0:5,:].to_html()))

tableau_frame2 = pd.DataFrame(tab,columns = ["9 moves","16 moves","14 moves"],
                               index = ["Expectimax Aléatoire"
                                        ,"Expectimax Pessimiste","Expectimax Optimiste"])
print("Number of played moves :")
display(HTML(tableau_frame2.iloc[0:3,:].to_html()))


Visited states :


,9 moves,16 moves,14 moves
No Pruning,4669.0,8162.0,5169.0
Pruning,3301.0,4468.0,2855.0
Expectimax Aléatoire,4509.0,6138.0,5117.0
Expectimax Pessimiste,19243.0,30391.0,41800.0
Expectimax Optimiste,15142.0,21298.0,15472.0


Number of played moves :


,9 moves,16 moves,14 moves
Expectimax Aléatoire,9.0,16.0,16.0
Expectimax Pessimiste,14.0,25.0,37.0
Expectimax Optimiste,9.0,16.0,16.0


### 4.1.4 Comparaison 

En comparant le nombre de movements nécessaires à la réussite, on voit que l'agent optimiste, comparé à l'agent aléatoire, est aussi bon, si ce n'est meilleur, car il esécute le movement avec le plus de chances de minimiser le nombre de mouvements. Par contre, l'agent pessimiste quant à lui exécute plus de movements que l'agent aléatoire, car il s'assure du mouvement avec un risque minimal. Plus d'états sont visités dans les deux derniers cas, car une recherche du maximum est effectuée avin de pondérer la probabilité avec une normalisation du score par rapport au maximum.



## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 